<a href="https://colab.research.google.com/github/oldaandozerskaya/DAGPap22/blob/main/sdp2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Read data

In [1]:
import pandas as pd
df = pd.read_csv("fake_papers_train_part_public.csv")
train_texts = list(df.text.values)
train_labels = list(df.fake.values)

In [2]:
df = pd.read_csv("fake_papers_test_public.csv")
test_texts = list(df.text.values)

In [3]:
#read generated texts, OPTIONAL
import pickle
with open('generated_texts/gpt2_kp20k.pickle', 'rb') as f:
  generated_texts = pickle.load(f)
train_texts.extend(generated_texts)
train_labels.extend([1 for el in generated_texts])
len(generated_texts)

3551

In [3]:
import collections
from collections import Counter
cnt = Counter(train_labels) #number of class examples
dict(cnt)

{0: 1686, 1: 3664}

#Model

In [4]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.metrics import f1_score, recall_score, precision_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [6]:
import numpy as np
!pip install imblearn
from imblearn.over_sampling import RandomOverSampler

f1_=0
num_folds = 3

for fold in range(num_folds):
        #split train/test
        papers_splitted = np.array_split(train_texts, num_folds)
        classes_splitted = np.array_split(train_labels, num_folds)
        test_fold_papers = papers_splitted[fold]
        test_fold_classes = classes_splitted[fold]

        train_fold_papers = []
        train_fold_classes = []
        for i, el in enumerate(papers_splitted):
          if i != fold:
            train_fold_papers.extend(el)
            train_fold_classes.extend(classes_splitted[i])


        train_fold_papers = list(train_fold_papers)
        train_fold_classes = list(train_fold_classes)
        
        #oversampling, OPTIONAL
        '''
        train_fold_papers = [[el] for el in train_fold_papers]
        train_fold_classes = [[el] for el in train_fold_classes]
        ros = RandomOverSampler(random_state=fold)
        train_fold_papers, train_fold_classes = ros.fit_resample(train_fold_papers, train_fold_classes)
        train_fold_papers = [el[0] for el in train_fold_papers]
        '''
        #class weighting, OPTIONAL
        '''
        class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_fold_classes),
                                        y = train_fold_classes                                                 
                                    )
        '''

        train_df = pd.DataFrame({"text": train_fold_papers, "labels": train_fold_classes})
        model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=True, \
                                        no_save = True, max_seq_length=128, sliding_window=True, train_batch_size = 16)

        model = ClassificationModel(
            "bert", "allenai/scibert_scivocab_cased", args=model_args, use_cuda=True
            #"roberta", "roberta-large", args=model_args, use_cuda=True#, weight = list(class_weights)
            #"deberta", "microsoft/deberta-large", args=model_args, use_cuda=True
        )

        model.train_model(train_df)
        predictions, raw_outputs = model.predict(list(test_fold_papers))
        f1 = f1_score(predictions, test_fold_classes, average = 'macro')
        print(f1)
        f1_+= f1
print(f'The avg F1 is approximately {f1_/3:.2f}.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

  0%|          | 0/3566 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/473 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/473 [00:00<?, ?it/s]

  0%|          | 0/1783 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

0.9510041244169982
The avg F1 is approximately 0.96.
